In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
from pathlib import Path

from matplotlib import pyplot as plt
# from tqdm import tqdm
from tqdm.notebook import tqdm
from pathlib import Path

### Load a classifier

In [ ]:
import torch
from collab_splats.utils.grouping import GroupingClassifier, GroupingConfig

# Path to the config for a trained model
base_dir = '/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features'
load_config = Path(base_dir) / "2025-07-25_074037" / "config.yml"

saved_model = Path(base_dir) / "grouping" / "checkpoints" / "grouping-classifier-v1.ckpt"

# if saved_model.exists():
#     print (f"Loading model from {saved_model}")
#     grouping_classifier = GroupingClassifier.load_from_checkpoint(saved_model)

#     grouping_classifier.load_pipeline()
# else:
grouping_config = GroupingConfig(
    segmentation_backend='mobilesamv2', 
    segmentation_strategy='object', 
    front_percentage=0.2, 
    iou_threshold=0.1, 
    num_patches=32,
    # lr=5e-5
)

grouping_classifier = GroupingClassifier(load_config=load_config, config=grouping_config)

# grouping_classifier.identities

# # Step 2: Load checkpoint state_dict only
# if saved_model.exists():
#     checkpoint = torch.load(saved_model)
#     state_dict = checkpoint['state_dict']
#     grouping_classifier.load_state_dict(state_dict, strict=False)

# # Step 3: Inject runtime pipeline & model
# grouping_classifier.load_pipeline()        # loads the NeRF pipeline at runtime
# grouping_classifier.load_segmentation()    # loads the segmentation backend

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Taichi] version 1.7.4, llvm 15.0.4, commit b4b956fd, linux, python 3.10.18


[I 09/23/25 20:53:14.970 194431] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


Memory bank loaded from /workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/grouping/memory_bank.pkl with 1994 masks


Process ForkProcess-23:
Process ForkProcess-18:
Process ForkProcess-2:
Process ForkProcess-30:
Process ForkProcess-1:
Process ForkProcess-19:
Process ForkProcess-24:
Process ForkProcess-6:
Process ForkProcess-4:
Process ForkProcess-10:
Process ForkProcess-29:
Process ForkProcess-9:
Process ForkProcess-8:
Process ForkProcess-17:
Process ForkProcess-20:
Process ForkProcess-25:
Process ForkProcess-5:
Process ForkProcess-26:
Process ForkProcess-11:
Process ForkProcess-3:
Process ForkProcess-32:
Process ForkProcess-15:
Process ForkProcess-27:
Process ForkProcess-31:
Process ForkProcess-16:
Process ForkProcess-7:
Process ForkProcess-21:
Process ForkProcess-22:
Process ForkProcess-28:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last)

### Associate masks

In [ ]:
grouping_classifier.create_masks()

In [ ]:
grouping_classifier.associate()

### Try pytorch lightning datamodule

Train identity embeddings to lift objects from 2d to 3d

In [ ]:
grouping_classifier.load_pipeline()

In [3]:
import wandb
from pytorch_lightning.loggers import WandbLogger

logger = WandbLogger(
    project="collab-splats", 
    name="grouping",
    log_model=False
)

grouping_classifier.setup()

Loading NeRF pipeline and model...


[20:53:23] Auto image downscale factor of 2                                                 ]8;id=60267;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=364419;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#484\484]8;;\

[20:53:55] use color only optimization with sigmoid activation                                         ]8;id=1874;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py\splatfacto.py]8;;\:]8;id=871001;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py#213\213]8;;\

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-25_074037/nerfstudio_models/step-00002
9999.ckpt

In [5]:
grouping_classifier.lift_segmentation(logger=logger)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


None



  | Name       | Type                | Params | Mode 
-----------------------------------------------------------
0 | pipeline   | VanillaPipeline     | 465 M  | eval 
1 | model      | RadegsFeaturesModel | 465 M  | eval 
2 | params     | ParameterDict       | 16.8 M | train
3 | classifier | Conv2d              | 27.9 K | train
4 | loss_fn    | CrossEntropyLoss    | 0      | train
-----------------------------------------------------------
16.9 M    Trainable params
465 M     Non-trainable params
482 M     Total params
1,929.316 Total estimated model params size (MB)
3         Modules in train mode
389       Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


SystemExit: 1

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7f257a7d7d60>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f23958de530, execution_count=5 error_before_exec=None error_in_exec=1 info=<ExecutionInfo object at 7f2379f0c670, raw_cell="grouping_classifier.lift_segmentation(logger=logge.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2B194.68.245.164/workspace/collab-splats/stage/test_grouping.ipynb#X15sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

### Random junk

In [ ]:
from collab_splats.utils.grouping import GroupingDataModule 

datamodule = GroupingDataModule(
    datamanager=grouping_classifier.pipeline.datamanager,
    mask_dir=grouping_classifier.associated_mask_dir,
    device="cuda",
    train_num_workers=0,
    val_num_workers=0,
)

In [ ]:
import numpy as np

grouping_classifier.eval()
logits = grouping_classifier(camera)

labels = logits.argmax(0).detach().cpu().numpy()
unique_labels = np.unique(labels)

print (unique_labels)

plt.imshow(labels)

plt.imshow(data['segmentation'])

In [ ]:
from torch.nn import CrossEntropyLoss

identities = identities.unsqueeze(0)
segmentation = data['segmentation'].unsqueeze(0).to(grouping_classifier.model.device)

CrossEntropyLoss(reduction="none")(identities, segmentation)

# grouping_classifier.loss_fn(identities, data['segmentation'])

### Try to map onto the mesh?

In [ ]:
import pickle
import open3d as o3d
from collab_splats.utils.mesh import features2vertex


mesh_dir = grouping_classifier.output_dir.parent / 'mesh'

mesh_path = mesh_dir / 'mesh.ply'
transforms_path = mesh_dir / 'transforms.pkl'

with open(transforms_path, 'rb') as f:
    transforms = pickle.load(f)

mesh = o3d.io.read_triangle_mesh(mesh_path)


In [ ]:
# Transform the means to the mesh
means = grouping_classifier.model.means.clone()
means = means @ transforms["mesh_transform"][:3, :3].T + transforms["mesh_transform"][:3, 3]

# Get the classes for each point
classes = grouping_classifier.per_gaussian_forward(grouping_classifier.identities)
classes = classes.argmax(-1).unsqueeze(-1)

Map to the mesh

In [ ]:
mesh_classes = features2vertex(
    mesh_vertices=mesh.vertices,
    points=means,
    features=classes,
    categorical=True
)

In [ ]:
import torch

# Create RGB colors for each unique class
unique_classes = torch.unique(mesh_classes)
n_classes = len(unique_classes)

# Generate distinct colors using HSV colorspace for better visual separation
import matplotlib.pyplot as plt
cmap = plt.get_cmap('tab10')  # or 'viridis', 'plasma', etc.

# Create color mapping
class_to_rgb = {}
for i, class_id in enumerate(unique_classes):
    color = cmap(i / max(1, n_classes - 1))  # Normalize to [0,1]
    class_to_rgb[class_id.item()] = torch.tensor(color[:3], dtype=torch.float32)  # RGB only

# Map classes to RGB colors
rgb_colors = torch.zeros(mesh_classes.shape[0], 3, dtype=torch.float32)
for i, class_id in enumerate(mesh_classes.squeeze()):
    rgb_colors[i] = class_to_rgb[class_id.item()]

rgb_colors

In [ ]:
import pyvista as pv

mesh = pv.read(mesh_path.as_posix())

print(mesh.n_points)
print(mesh.n_cells)
print(mesh.bounds)

image = mesh.plot(
    scalars=rgb_colors, 
    rgb=True,
    screenshot=True
)

plt.imshow(image)